## This notebook contains steps for configuring an instance of SQL Server running in a docker container.  It uses PowerShell functions in the DBATools module.  The following script will check if DBATools is installed.  If it is not, it will install it.  If it is installed, it will be updated to the latest version.

In [3]:
$exists = (Get-InstalledModule -Name DBATools).Name;
$exists;

if(!$exists)
{
    Install-Module -Name DBATools;   
}
else
{
    Update-DbaTools;
}

## Once DBATools is installed, you are ready to start.  The first step sets max server memory (mb) to a recommended value.

In [2]:
$path = 'stuff';
$maxmemory = "`$cred = Get-Credential;
Set-DbaMaxMemory -SqlInstance 'localhost,1436' -SqlCredential $cred;"

$exists = Test-Path $path;

if($exists)
{
    Remove-Item -Path $path;
}

New-Item -Path $path -ItemType file;
Add-Content -Path $path -Value $maxmemory;



    Directory: C:\Users\fgill


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
-a----       12/18/2019   2:04 PM              0 stuff                                                                 


